# Flaw Engine: Cloud Trainer (Gideon-gg101 Edition) 🚀

This notebook turns this Colab instance into a **Distributed Worker** for the Flaw Chess Engine.

### 🏁 Setup Options:
Run **ONLY ONE** of the setup options below.

## 1.1 Option A: Setup via GitHub (Recommended)
Use this if you have pushed your code to GitHub.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

# 1. Clean up everything
!rm -rf /content/flaw
print("Starting Clone...")

# 2. Clone from GitHub
!git clone {GITHUB_URL} /content/flaw

# 3. Check what was created
import os
print("Checking /content/ directory:")
!ls -p /content/

if os.path.exists('/content/flaw'):
    print("\n✅ GitHub Sync Successful!")
    %cd /content/flaw
else:
    print("\n❌ FAILED to clone.")
    print("HINT: If your repository is PRIVATE, you must make it PUBLIC for this link to work.")

## 1.2 Option B: Setup via Manual ZIP
Use this if GitHub is giving you path errors. Upload `flaw_cloud_bundle.zip` to the file explorer on the left.

In [ ]:
!rm -rf /content/flaw
!mkdir -p /content/flaw
!unzip -o /content/flaw_cloud_bundle.zip -d /content/flaw

import os
if os.path.exists('/content/flaw'):
    print("✅ Unzip Successful!")
    %cd /content/flaw
else:
    print("❌ FAILED to find ZIP. Did you upload 'flaw_cloud_bundle.zip'?")

## 2. Environment & Build
Run this to compile the engine.

In [ ]:
!apt-get install -y cmake g++ > /dev/null
!pip install requests torch > /dev/null

import os
if not os.path.exists('/content/flaw'):
    raise Exception("❌ Stop! You must run Section 1 first to download the code.")

%cd /content/flaw
!mkdir -p core/build
%cd core/build
!cmake ..
!make -j$(nproc)

# Move the built library to the root
!cp flaw_core*.so ../../

%cd /content/flaw
print("\n✅ Build Complete! Current items:")
!ls -F

## 3. Start Training
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import os
if not os.path.exists('/content/flaw/flaw_core.so') and not os.path.exists('/content/flaw/flaw_core.cpython-310-x86_64-linux-gnu.so'):
    print("⚠️ Warning: flaw_core.so not found at root. Checking if build succeeded...")

!export PYTHONPATH=$PYTHONPATH:/content/flaw && python3 -m ai.selfplay_worker --master {MASTER_URL}